In [ ]:
# 설치
! pip install smolagents
# 마지막 릴리스 대신 소스에서 설치하려면 위 명령을 주석 처리하고 다음 명령의 주석을 해제하십시오.
# ! pip install git+https://github.com/huggingface/smolagents.git

# OpenTelemetry로 실행 검사하기

> [!TIP]
> 에이전트 빌드가 처음이라면 먼저 [에이전트 소개](https://huggingface.co/docs/smolagents/main/ko/tutorials/../conceptual_guides/intro_agents)와 [smolagents 둘러보기](https://huggingface.co/docs/smolagents/main/ko/tutorials/../guided_tour)를 읽어보세요.

## 에이전트 실행을 기록해야 하는 이유

에이전트 실행은 디버깅하기 복잡합니다.

에이전트 워크플로는 [설계상 예측 불가능](https://huggingface.co/docs/smolagents/main/ko/tutorials/../conceptual_guides/intro_agents)하기 때문에 실행이 제대로 되었는지 확인하기 어렵습니다(예측 가능했다면 기존 코드를 사용했을 것입니다).

그리고 실행을 검사하는 것도 어렵습니다. 다단계 에이전트는 콘솔을 로그로 빠르게 채우는 경향이 있으며 대부분의 오류는 LLM이 더 나은 코드나 도구 호출을 작성하여 다음 단계에서 자동 수정하는 "LLM 바보" 종류의 오류입니다.

따라서 나중에 검사하고 모니터링하기 위해 프로덕션에서 에이전트 실행을 기록하기 위해 계측을 사용하는 것이 필요합니다!

에이전트 실행을 계측하기 위해 [OpenTelemetry](https://opentelemetry.io/) 표준을 채택했습니다.

즉, 일부 계측 코드를 실행한 다음 에이전트를 정상적으로 실행하면 모든 것이 플랫폼에 기록됩니다. 다음은 다양한 OpenTelemetry 백엔드를 사용하여 이 작업을 수행하는 방법에 대한 몇 가지 예입니다.

플랫폼에서 보이는 모습은 다음과 같습니다.

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/smolagents/inspect_run_phoenix.gif"/>
</div>

## Arize AI Phoenix로 원격 측정 설정

먼저 필요한 패키지를 설치합니다. 여기서는 로그를 수집하고 검사하는 좋은 솔루션인 [Arize AI의 Phoenix](https://github.com/Arize-ai/phoenix)를 설치하지만 이 수집 및 검사 부분에 사용할 수 있는 다른 OpenTelemetry 호환 플랫폼이 있습니다.

```shell
pip install 'smolagents[telemetry,toolkit]'
```

그런 다음 백그라운드에서 수집기를 실행합니다.

```shell
python -m phoenix.server.main serve
```

마지막으로 에이전트를 추적하고 추적을 Phoenix 기본 엔드포인트로 보내도록 `SmolagentsInstrumentor`를 설정합니다.

In [ ]:
from phoenix.otel import register
from openinference.instrumentation.smolagents import SmolagentsInstrumentor

register()
SmolagentsInstrumentor().instrument()

그런 다음 에이전트를 실행할 수 있습니다!

In [ ]:
from smolagents import (
    CodeAgent,
    ToolCallingAgent,
    WebSearchTool,
    VisitWebpageTool,
    InferenceClientModel,
)

model = InferenceClientModel()

search_agent = ToolCallingAgent(
    tools=[WebSearchTool(), VisitWebpageTool()],
    model=model,
    name="search_agent",
    description="웹 검색을 수행할 수 있는 에이전트입니다.",
)

manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[search_agent],
)
manager_agent.run(
    "미국이 2024년 성장률을 유지한다면 GDP가 두 배가 되는 데 몇 년이 걸릴까요?"
)

짜잔!
그런 다음 `http://0.0.0.0:6006/projects/`로 이동하여 실행을 검사할 수 있습니다!

<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/smolagents/inspect_run_phoenix.png">

CodeAgent가 관리형 ToolCallingAgent(관리형 에이전트는 CodeAgent일 수도 있음)를 호출하여 미국 2024년 성장률에 대한 웹 검색을 실행하도록 요청한 것을 볼 수 있습니다. 그런 다음 관리형 에이전트가 보고서를 반환했고 관리자 에이전트가 이를 기반으로 경제 배가 시간을 계산했습니다! 멋지죠?

## Langfuse로 원격 분석 설정

이 부분에서는 `SmolagentsInstrumentor`를 사용하여 Hugging Face **smolagents**를 **Langfuse**로 모니터링하고 디버깅하는 방법을 보여줍니다.

> **Langfuse란 무엇인가요?** [Langfuse](https://langfuse.com)는 LLM 엔지니어링을 위한 오픈 소스 플랫폼입니다. AI 에이전트에 대한 추적 및 모니터링 기능을 제공하여 개발자가 제품을 디버깅, 분석 및 최적화하는 데 도움을 줍니다. Langfuse는 기본 통합, OpenTelemetry 및 SDK를 통해 다양한 도구 및 프레임워크와 통합됩니다.

### 1단계: 종속성 설치

In [ ]:
%pip install smolagents
%pip install opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents

### 2단계: 환경 변수 설정

Langfuse API 키를 설정하고 OpenTelemetry 엔드포인트를 구성하여 Langfuse로 추적을 보냅니다. [Langfuse Cloud](https://cloud.langfuse.com)에 가입하거나 [Langfuse를 자체 호스팅](https://langfuse.com/self-hosting)하여 Langfuse API 키를 받으십시오.

또한 [Hugging Face 토큰](https://huggingface.co/settings/tokens)(`HF_TOKEN`)을 환경 변수로 추가하십시오.

In [ ]:
import os
import base64

LANGFUSE_PUBLIC_KEY="pk-lf-..."
LANGFUSE_SECRET_KEY="sk-lf-..."
LANGFUSE_AUTH=base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU 데이터 지역
# os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # 미국 데이터 지역
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

# Hugging Face 토큰
os.environ["HF_TOKEN"] = "hf_..."

### 3단계: `SmolagentsInstrumentor` 초기화

애플리케이션 코드 전에 `SmolagentsInstrumentor`를 초기화합니다. `tracer_provider`를 구성하고 스팬 프로세서를 추가하여 Langfuse로 추적을 내보냅니다. `OTLPSpanExporter()`는 환경 변수의 엔드포인트와 헤더를 사용합니다.

In [ ]:
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

### 4단계: smolagent 실행

In [ ]:
from smolagents import (
    CodeAgent,
    ToolCallingAgent,
    WebSearchTool,
    VisitWebpageTool,
    InferenceClientModel,
)

model = InferenceClientModel(
    model_id="deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"
)

search_agent = ToolCallingAgent(
    tools=[WebSearchTool(), VisitWebpageTool()],
    model=model,
    name="search_agent",
    description="웹 검색을 수행할 수 있는 에이전트입니다.",
)

manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[search_agent],
)
manager_agent.run(
    "Langfuse를 사용하여 smolagents가 사용자 피드백이나 사용 가능한 재료를 기반으로 레시피를 동적으로 조정하는 것과 같은 다단계 작업을 실행할 때 추론 및 의사 결정을 모니터링하고 개선하는 방법은 무엇인가요?"
)

### 5단계: Langfuse에서 추적 보기

에이전트를 실행한 후 [Langfuse](https://cloud.langfuse.com)에서 smolagents 애플리케이션이 생성한 추적을 볼 수 있습니다. LLM 상호 작용의 자세한 단계를 볼 수 있으며 이는 AI 에이전트를 디버깅하고 최적화하는 데 도움이 될 수 있습니다.

![smolagents 예제 추적](https://langfuse.com/images/cookbook/integration-smolagents/smolagent_example_trace.png)

_[Langfuse의 공개 예제 추적](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/ce5160f9bfd5a6cd63b07d2bfcec6f54?timestamp=2025-02-11T09%3A25%3A45.163Z&display=details)_